In [6]:
from pathlib import Path

dir_prefix = r'D:\CodeBase\Clara\chatbot\novel_writer'
file_name = dir_prefix + r'\train_data\shendiaoxialv.txt'
file_path = Path(file_name)
print(file_path) 
with open(file_path, 'r', encoding='gbk') as f:
    text = f.read()
print(f'number of total text {len(text)}')
charset = set(text)
print(f'number of total chars {len(charset)}')
print(text[:100])


D:\CodeBase\Clara\chatbot\novel_writer\train_data\shendiaoxialv.txt
number of total text 1021857
number of total chars 4100

------------------------------------------
“金庸作品集”新序
　　小说是写给人看的。小说的内容是人。
　　小说写一个人、几个人、一群人、或成千成万人的性格


In [2]:
encoded_map = {char:i for i, char in enumerate(charset)}
decode_map = {i:char for i, char in enumerate(charset)}
encode = lambda input_text: [encoded_map[x] for x in input_text]
decode = lambda code: ''.join([decode_map[x] for x in code])


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1021857]) torch.int64
tensor([ 440,  255,  255,  255,  255,  255,  255,  255,  255,  255,  255,  255,
         255,  255,  255,  255,  255,  255,  255,  255,  255,  255,  255,  255,
         255,  255,  255,  255,  255,  255,  255,  255,  255,  255,  255,  255,
         255,  255,  255,  255,  255,  255,  255,  440, 2522, 1052, 1516,  286,
         481, 1246, 2990, 1626, 2957,  440, 3468, 3468, 3763, 4030,  123, 2437,
        3848,  947, 4064,  909, 1659, 3763, 4030,  909, 2230, 1423,  123,  947,
        1659,  440, 3468, 3468, 3763, 4030, 2437, 1555, 1099,  947,  344,  604,
        1099,  947,  344, 1555, 1812,  947,  344, 1358, 2762, 3713, 2762,  823,
         947,  909,  563, 2624])


In [8]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(train_data.shape, val_data.shape)

torch.Size([919671]) torch.Size([102186])


In [16]:
block_size = 10
batch_size = 4

def get_batch(split:str):
    data = train_data if split == 'train' else val_data
    train_index = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in train_index])
    y = torch.stack([data[i+1:i+block_size+1] for i in train_index])
    return x, y

input, output = get_batch('train')
print('input:')
print(input.shape)
print(input)

print('output')
print(output.shape)
print(output)

for batch in range(batch_size):
    for pos in range(block_size):
        print(f"input is {decode(input[batch, :pos + 1].tolist())} output is {decode([output[batch, pos].item()])}")

input:
torch.Size([4, 10])
tensor([[2840, 1620, 2981, 1648, 2219, 2122, 2099, 2522,   34,  638],
        [2693,  856, 2139, 1297, 1690, 1450, 1648, 1304,  523, 1869],
        [ 440, 3468, 3468, 1661, 2711, 4048,  283, 3993, 3081,  429],
        [3507, 3192, 3129, 3502, 1648, 1555, 1943, 1922, 1404, 1659]])
output
torch.Size([4, 10])
tensor([[1620, 2981, 1648, 2219, 2122, 2099, 2522,   34,  638,  394],
        [ 856, 2139, 1297, 1690, 1450, 1648, 1304,  523, 1869, 1750],
        [3468, 3468, 1661, 2711, 4048,  283, 3993, 3081,  429, 1828],
        [3192, 3129, 3502, 1648, 1555, 1943, 1922, 1404, 1659, 1732]])
input is 宫 output is 房
input is 宫房 output is 舍
input is 宫房舍 output is ，
input is 宫房舍， output is 寻
input is 宫房舍，寻 output is 思
input is 宫房舍，寻思 output is ：
input is 宫房舍，寻思： output is “
input is 宫房舍，寻思：“ output is 我
input is 宫房舍，寻思：“我 output is 暗
input is 宫房舍，寻思：“我暗 output is 中
input is 得 output is 自
input is 得自 output is 兄
input is 得自兄 output is 长
input is 得自兄长 output is 亲
input is 得自